# imports

In [1]:
import random, os, math, itertools, re, unicodedata
from pathlib import Path
from typing import List, Tuple, Dict, Any
import torch.nn.functional as F

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import Tensor
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

from datasets import (
    load_dataset,
    load_from_disk,
    DatasetDict,
    Dataset as HFDataset,
    concatenate_datasets,          
)
from transformers import AutoTokenizer, get_linear_schedule_with_warmup, AutoModelForCausalLM
from rouge_score import rouge_scorer

/Users/dotsenko.a.v/yandex/sprint2-project/sp2proj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Глобальные переменные

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
CSV_PATH = './data/training.1600000.processed.noemoticon.csv' #"./data/training.1600000.processed.noemoticon.csv"
ROUGE = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
URL_RE      = re.compile(r'https?://\S+|www\.\S+', flags=re.IGNORECASE)
MENTION_RE  = re.compile(r'@\w+')
HASHTAG_RE  = re.compile(r'#(\w+)')
NUM_WORKERS = 0
COMPARISON_DS_SIZE = 100 # количество записей из датасета для сравнения трансформера и lstm
MAX_GEN_LEN = 20
BEST_MODEL_PATH = f"./models/best_models/final_model.pt"


# Датасет

In [3]:
def read_dataset(csv_path: str,
                 tokenizer: AutoTokenizer,
                 max_length: int = 128,
                 nrows: int | None = None) -> HFDataset:
    """
    Возвращает HuggingFace-Dataset с колонками:
        - input_ids (list[int])
        - attention_mask (list[int])
    """
    col_names = ["target", "ids", "date", "flag", "user", "text"]
    df_raw = pd.read_csv(
        csv_path,
        header=None,
        names=col_names,
        encoding="latin1",
        dtype=str,
        usecols=["text"],
        na_values=["NO_QUERY"],
        keep_default_na=False,
        on_bad_lines="skip",
        nrows=nrows,
    )
    df_raw["text"] = df_raw["text"].apply(clean_tweet)
    cleaned_texts = df_raw["text"].tolist()
    dataset = HFDataset.from_pandas(df_raw)
    tokenized = dataset.map(
        lambda batch: tokenizer(batch["text"],
                               truncation=True,
                               max_length=max_length,
                               padding=False,
                               return_attention_mask=True),
        batched=True,
        remove_columns=["text"]
    )
    return tokenized, cleaned_texts

def split_hf_dataset(hf_dataset: HFDataset,
                     train_ratio: float = 0.80,
                     val_ratio:   float = 0.10,
                     test_ratio:  float = 0.10,
                     seed: int = 42) -> DatasetDict:
    '''
    Принимает токенизированный Dataset и возвращает
    три датасета:
        - train
        - validation
        - test

    Параметры:
        train_ratio, val_ratio, test_ratio – доли.
        seed – фиксирует случайность, чтобы результаты были воспроизводимы.
    '''
    if not 0 < train_ratio < 1 or not 0 < val_ratio < 1 or not 0 < test_ratio < 1:
        raise ValueError("Все доли должны лежать в (0,1)")
    split1 = hf_dataset.train_test_split(test_size=1.0 - train_ratio, seed=seed)
    train_ds = split1['train']
    rest = split1['test']

    val_rel = val_ratio / (val_ratio + test_ratio)
    split2 = rest.train_test_split(test_size=1.0 - val_rel, seed=seed)

    val_ds = split2['train']
    test_ds = split2['test']

    return DatasetDict({
        'train': train_ds,
        'validation': val_ds,
        'test': test_ds
    })


def remove_emoji(text: str) -> str:
    """
    Удаляет все символы, у которых Unicode-категория 
    начинается с 'S' (Symbol) или 'C' (Other, в т.ч. 
    контрольные символы).
    """
    return ''.join(ch for ch in text
                   if not (unicodedata.category(ch).startswith('S') or
                           unicodedata.category(ch).startswith('C')))

def clean_tweet(text_as_is: str) -> str:
    """
    text_as_is - твит
    
    Убирает:
        ссылки
        упоминания
        эмодзи
        лишние пробелы

    Возвращает: очищенный твит
    """
    if not isinstance(text_as_is, str):
        return ''

    text = text_as_is.lower()
    text = URL_RE.sub('', text)
    text = MENTION_RE.sub('', text)
    text = HASHTAG_RE.sub(r'\1', text)
    text = remove_emoji(text) 
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def make_val_loader(
    tokenizer: AutoTokenizer,
    eos_id: int,
    device: torch.device,
    nrows: int | None = None,
) -> Tuple[DataLoader, DatasetDict, List[str]]:
    
    VAL_BATCH_SIZE = 10 # размер батча для сравнения трансформера и lstm
    """
    Читает датасет, делит его на splits и возвращает
    val_loader – DataLoader, готовый к использованию LSTM-моделью;
    splits – словарь с HF-сплитами (можно взять validation-датасет);
    val_texts – список чистых твитов (строк), которые нужны трансформеру.
    """
    hf_tokenized, cleaned_texts = read_dataset(CSV_PATH,
                                               tokenizer, 
                                               max_length=tokenizer.model_max_length,
                                               nrows=nrows)
    val_ds = TweetDataset(hf_tokenized, eos_id=eos_id)
    val_loader = DataLoader(
        val_ds,
        batch_size=VAL_BATCH_SIZE,
        shuffle=False,
        collate_fn=token_collate_fn,
        num_workers=NUM_WORKERS,
        pin_memory=device.type == "cuda",
    )
    val_indices = hf_tokenized["__index__"] if "__index__" in hf_tokenized.features else None
    if val_indices is not None:
        # Если у HF-датасета есть поле __index__, используем его.
        val_texts = [cleaned_texts[i] for i in val_indices]
    else:
        # Если индексов нет (в старых версиях), просто берём первые N
        # элементов, где N = len(splits["validation"]).
        val_texts = cleaned_texts[:len(hf_tokenized)]
    return val_loader, hf_tokenized, val_texts, 

class TweetDataset(Dataset):
    def __init__(self, hf_dataset: HFDataset, eos_id: int):
        self.input_ids = [ex["input_ids"] for ex in hf_dataset]
        self.attn_mask = [ex["attention_mask"] for ex in hf_dataset]
        self.eos_id = eos_id

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return (torch.tensor(self.input_ids[idx], dtype=torch.long),
                torch.tensor(self.attn_mask[idx], dtype=torch.long))
    

def token_collate_fn(batch):
    # batch = [(ids, mask), …]
    ids, masks = zip(*batch)
    ids = pad_sequence(ids, batch_first=True, padding_value=0)
    masks = pad_sequence(masks, batch_first=True, padding_value=0)
    # Для LSTM нам нужны также `labels` – сдвинутые на 1 токен
    labels = ids.clone()
    labels[:, :-1] = ids[:, 1:]
    labels[:, -1] = -100          # ignore last token
    return {"input_ids": ids,
            "attention_mask": masks,
            "labels": labels}

# Вспомогательные функции

In [4]:
def get_eos_id(tokenizer: AutoTokenizer) -> int:
    """Возвращает id конца последовательности"""
    if tokenizer.eos_token_id is not None:
        return tokenizer.eos_token_id
    if tokenizer.sep_token_id is not None:
        return tokenizer.sep_token_id
    raise ValueError("Tokenizer has neither eos_token nor sep_token.")

def rouge_l_f1(ref: str, hyp: str) -> float:
    return ROUGE.score(ref, hyp)['rougeL'].fmeasure

def top_p_filtering(logits: Tensor, top_p: float = 0.9) -> Tensor:
    """
    Оставляем только те токены, чья суммарная вероятность (по убыванию) ≤ top_p.
    Возвращаем logits, где остальные токены заменены на -inf.
    """
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

    # Оставляем токены, пока кумулятивная вероятность ≤ top_p
    sorted_indices_to_keep = cumulative_probs <= top_p
    # гарантируем, что как минимум один токен остаётся (первый)
    sorted_indices_to_keep[..., 0] = 1

    # Маска: -inf для токенов, которые выкинули
    mask = torch.full_like(logits, float("-inf"))
    mask.scatter_(
        dim=-1,
        index=sorted_indices,
        src=sorted_logits.masked_fill(~sorted_indices_to_keep, float("-inf")),
    )
    return mask

def set_seed(seed: int = 42):
    '''
    Позволяет выполнять детерминированный запуск
    '''
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# LSTM

In [5]:
class LSTMWordGenerator(nn.Module):
    """
    Train: Embedding → (Bi)LSTM → Linear (vocab size)
    Inference: Embedding → LSTM → Linear (vocab size)
    """
    def __init__(self,
                 vocab_size: int,
                 embed_dim: int = 256,
                 hidden_dim: int = 512,
                 num_layers: int = 2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
        )
        lstm_out_dim = hidden_dim
        self.fc = nn.Linear(lstm_out_dim, vocab_size)

    def forward(self,
                input_ids: torch.Tensor,
                attention_mask: torch.Tensor,
                labels: torch.Tensor | None = None):
        '''
        Возвращает логиты и ошибку
        '''
        embeds = self.embedding(input_ids)               # (B, L, D)

        # защита от нулевых длин
        lengths = attention_mask.sum(dim=1).cpu()        # (B,)
        # Если в батче есть полностью пустые примеры (length == 0),
        # заменяем 0 на 1, чтобы pack_padded_sequence не падала.
        if (lengths == 0).any():
            lengths = lengths.clone()
            lengths[lengths == 0] = 1

        packed = pack_padded_sequence(
            embeds, lengths, batch_first=True, enforce_sorted=False
        )
        packed_out, _ = self.lstm(packed)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)

        logits = self.fc(out)

        if labels is None:
            return {'logits': logits}
        else:
            loss_fn = nn.CrossEntropyLoss(ignore_index=-100)
            loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
            return {'logits': logits, 'loss': loss}

    @torch.no_grad()
    def generate_one_word(self, 
                            text_prompt: str, 
                            tokenizer: AutoTokenizer, 
                            eos_id: int | None = None) -> str:
        """
        Генерирует одно слово после заданного текстового префикса
        """
        device = next(self.parameters()).device
        self.eval()

        # Токенизируем текст и получаем IDs
        inputs = tokenizer(text_prompt, return_tensors="pt").to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs.get("attention_mask", None)

        # Выполняем прямой проход через модель
        outputs = self.forward(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs["logits"][:, -1, :]  # Берем последние токены

        # Предсказываем следующий токен
        next_token = torch.argmax(logits, dim=-1)[0].item()

        # Преобразуем токен обратно в текст
        word = tokenizer.decode([next_token])
        return word

    @torch.no_grad()
    def generate_n_words(self, 
                            text_prompt: str, 
                            n: int, 
                            tokenizer: AutoTokenizer, 
                            eos_id: int | None = None, 
                            do_sampling: bool = False, 
                            temperature: float = 1.0, 
                            top_p: float = 0.9) -> str:
        """
        Генерирует N новых слов после заданного текстового префикса.
        """
        device = next(self.parameters()).device
        self.eval()

        # Токенизируем входной текст
        inputs = tokenizer(text_prompt, return_tensors="pt").to(device)
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs.get("attention_mask", None)

        # Начальные токены
        current_ids = input_ids

        with torch.no_grad():
            for _ in range(n):
                # Создаем внимание на всю длину текущих токенов
                att_mask = torch.ones_like(current_ids, device=device)

                # Прогоняем через модель
                outputs = self.forward(input_ids=current_ids, attention_mask=att_mask)
                logits = outputs["logits"][:, -1, :]  # Последние токены

                if do_sampling:
                    # Применяем температуру
                    logits /= max(temperature, 1e-8)

                    # Отсекаем редкие токены по Top-P
                    filtered_probs = F.softmax(top_p_filtering(logits.squeeze(), top_p=top_p), dim=-1)
                    next_token = torch.multinomial(filtered_probs, num_samples=1).unsqueeze(0)
                else:
                    # Просто выбираем самый вероятный токен
                    next_token = torch.argmax(logits, dim=-1, keepdim=True)

                # Добавляем токен в последовательность
                current_ids = torch.cat((current_ids, next_token), dim=1)

                # Проверка на достижение символа конца строки
                if eos_id is not None and next_token.item() == eos_id:
                    break

        # Переводим токены обратно в текст
        result_text = tokenizer.decode(current_ids.squeeze().tolist())
        return result_text[len(text_prompt):].strip()  # Убираем оригинальный текст и лишние пробелы

    def generate_one_sample(self,
                            prompt_ids: List[int],
                            eos_id: int,
                            num) -> List[int]:
        """
        Жадная генерация (по-умолчанию).  
        Останавливается, когда сгенерирован `eos_id` или
        достигнут `max_gen_len`.

        prompt_ids - Список токенов-промпта (может уже содержать `eos_id`).
        eos_id - ID токена конца предложения.
        к уже существующему промпту.

        Возвращает Полный список токенов (промпт+сгенерированное продолжение).
        """
        device = next(self.parameters()).device
        self.eval()

        # Защита от пустого промпта (аналогично generate)
        if not prompt_ids:
            return [] if eos_id is None else [eos_id]

        # Приводим промпт к тензору формы (1, L)
        generated = torch.tensor(prompt_ids,
                                 dtype=torch.long,
                                 device=device).unsqueeze(0)   # (1, L)

        with torch.no_grad():
            for _ in range(MAX_GEN_LEN):
                # учитывает все уже сгенерированные токены
                attn_mask = torch.ones_like(generated, device=device)

                out = self(input_ids=generated,
                           attention_mask=attn_mask)          # logits: (1, cur_len, vocab)
                next_token_logits = out['logits'][:, -1, :]      # (1, vocab)

                # Жадный выбор (можно заменить на sampling/beam-search)
                next_token = torch.argmax(next_token_logits, dim=-1)  # (1)

                # Добавляем выбранный токен к последовательности
                generated = torch.cat([generated,
                                      next_token.unsqueeze(-1)], dim=1)

                # Прерываем, если получили EOS
                if eos_id is not None and next_token.item() == eos_id:
                    break

        # Возвращаем чистый список int-ов (без batch-измерения)
        return generated.squeeze().tolist()        
    
    def tokens_to_words(self, 
                        gen_ids: List[int],
                        eos_id: int,
                        tokenizer: AutoTokenizer) -> List[str]:
        '''
        Получает сгенерированные айдишнки
        Берет полезную часть (до паддинга)
        На выходе декодированный текст
        '''
        # Обрезаем сгенерированный токен-список до EOS (если он есть) ----
        if eos_id in gen_ids:
            gen_ids = gen_ids[:gen_ids.index(eos_id)]

        # Декодируем оба текста ----
        gen_words = tokenizer.decode(gen_ids, skip_special_tokens=True)
        return gen_words
    
    def words_to_tokens(
        self,
        tweet: str,
        tokenizer: AutoTokenizer,
        add_special_tokens: bool = False) -> List[str]:
        """
        Преобразует один твит в список токенов.
        """
        enc = tokenizer(tweet,
                         truncation=True,
                         max_length=MAX_GEN_LEN,
                         add_special_tokens=add_special_tokens,
                         padding=False)
        prompt_ids = [ids for ids in enc['input_ids']]
        return prompt_ids

    def complete(self, *, 
                 text: str,
                 eos_id: int,
                 tokenizer: AutoTokenizer,
                 add_special_tokens=False,
                 preprocess: bool = True) -> str:
        
        prompt_ids = self.words_to_tokens(text, tokenizer, add_special_tokens, MAX_GEN_LEN)
        generated_ids = self.generate(prompt_ids=prompt_ids,
                                      eos_id=eos_id,
                                      do_sampling=True,
                                      temperature=0.7,
                                      top_p=0.8)
        generated_text = self.tokens_to_words(generated_ids, eos_id, tokenizer)
        return generated_text

    def generate(self,
                prompt_ids: List[int],
                eos_id: int,
                do_sampling: bool = False,
                temperature: float = 1.0,
                top_p: float = 0.9) -> List[int]:
        """
        Универсальный генератор, поддерживает:
        * greedy (do_sampling=False)
        * sampling с temperature / top-p (do_sampling=True)
        """
        device = next(self.parameters()).device
        self.eval()

        # Защита от полностью пустого промпта
        if not prompt_ids:                     # ничего не генерируем
            # Возвращаем либо пустой список, либо [eos_id] – выбираем
            # вариант, который проще обрабатывается дальше.
            return [] if eos_id is None else [eos_id]

        # (1, L) – уже tokenы промпта
        generated = torch.tensor(prompt_ids,
                                 dtype=torch.long,
                                 device=device).unsqueeze(0)   # (1, L)

        with torch.no_grad():
            for _ in range(MAX_GEN_LEN):
                attn_mask = torch.ones_like(generated, device=device)

                out = self(input_ids=generated,
                           attention_mask=attn_mask)  # logits (1, cur_len, vocab)
                next_logits = out["logits"][:, -1, :]                     # (1, vocab)

                if do_sampling:
                    # temperature
                    logits = next_logits / max(temperature, 1e-8)

                    # top-p
                    logits = top_p_filtering(logits.squeeze(0), top_p=top_p).unsqueeze(0)

                    # выбор из распределения
                    probs = F.softmax(logits, dim=-1)
                    next_token = torch.multinomial(probs, num_samples=1)   # (1,1)
                else:            # greedy
                    next_token = torch.argmax(next_logits, dim=-1, keepdim=True)  # (1,1)

                generated = torch.cat([generated, next_token], dim=1)

                # EOS?
                if eos_id is not None and next_token.item() == eos_id:
                    break

        return generated.squeeze().tolist()

# Сетап обучения LSTM

In [6]:
def complete_text(
    prompt_ids: List[int],
    model: AutoModelForCausalLM,
    eos_id: int,
    do_sampling: bool = False,
    temperature: float = 1.0,
    top_p: float = 0.9,
) -> List[int]:
    device = next(model.parameters()).device
    input_ids = torch.tensor([prompt_ids], dtype=torch.long, device=device)

    gen_kwargs = {
        "max_new_tokens": MAX_GEN_LEN,
        "eos_token_id": eos_id,
        "do_sample": do_sampling,
    }
    if do_sampling:
        gen_kwargs.update(
            {"temperature": temperature, "top_p": top_p}
        )
    out = model.generate(input_ids, **gen_kwargs)          # (1, prompt+generated)
    generated = out[0][input_ids.shape[-1] :].tolist()    # только сгенерированное
    return generated

def evaluate_on_loader(model: nn.Module,
                        loader: DataLoader,
                        tokenizer: AutoTokenizer,
                        device: torch.device,
                        eos_id: int,
                        fraction: float = 0.5,
                        use_sampling: bool = False,
                        temperature: float = 0.7,
                        top_p: float = 0.9,
                        verbose: bool = False) -> Tuple[float, float, List[float] | None]:
    """
    Возвращает (perplexity, avg_rougeL, per_example_scores|None)
    Оценка одинаково работает для LSTM и для трансформера,
    при условии, что модель реализует метод `generate_one_sample`
    (у LSTM он уже есть, у трансформера реализуем «обёртку» ниже).
    """
    model.eval()
    total_nll, total_tokens = 0.0, 0
    rouge_sum, n_examples = 0.0, 0
    per_example = [] if verbose else None

    with torch.no_grad():
        for batch in loader:
            # perplexity
            input_ids = batch["input_ids"].to(device)
            attn_mask = batch["attention_mask"].to(device)
            labels    = batch["labels"].to(device)

            out = model(input_ids=input_ids,
                        attention_mask=attn_mask,
                        labels=labels)
            loss = out["loss"]
            # количество токенов без паддинга:
            n_tok = attn_mask.sum().item()
            total_nll += loss.item() * n_tok
            total_tokens += n_tok

            #  ROUGE 
            ids_np = input_ids.cpu().numpy()
            for i in range(ids_np.shape[0]):
                seq = ids_np[i].tolist()
                # убираем PAD и EOS
                if 0 in seq:
                    seq = seq[:seq.index(0)]
                if eos_id in seq:
                    seq = seq[:seq.index(eos_id)]

                if not seq:
                    continue

                split = int(len(seq) * fraction)
                prompt_ids = seq[:split]
                ref_ids    = seq[split:]

                # генерация (можно использовать единый интерфейс)
                if isinstance(model, LSTMWordGenerator):
                    gen_ids = model.generate(prompt_ids=prompt_ids,
                                            eos_id=eos_id,
                                            do_sampling=use_sampling,
                                            temperature=temperature,
                                            top_p=top_p)
                else:   # трансформер
                    gen_ids = complete_text(prompt_ids=prompt_ids,
                                            tokenizer=tokenizer,
                                            model=model,
                                            eos_id=eos_id,
                                            do_sampling=use_sampling,
                                            temperature=temperature,
                                            top_p=top_p)
                # декодируем
                gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True)
                ref_text = tokenizer.decode(ref_ids, skip_special_tokens=True)

                rouge_f = rouge_l_f1(ref_text, gen_text)
                rouge_sum += rouge_f
                n_examples += 1
                if verbose:
                    per_example.append(rouge_f)

    perplexity = math.exp(total_nll / total_tokens) if total_tokens > 0 else float('inf')
    avg_rouge = rouge_sum / n_examples if n_examples > 0 else 0.0
    return perplexity, avg_rouge, per_example

def grid_search(
    param_grid: Dict[str, List[Any]],
    base_train_loader: DataLoader,
    base_val_loader: DataLoader,
    base_test_loader: DataLoader,
    tokenizer: AutoTokenizer,
    device: torch.device,
    eos_id: int,
    results_dir: str = "models") -> pd.DataFrame:
    """
    Перебирает все комбинации параметров, обучает модель,
    сохраняет лучшую модель и графики, возвращает таблицу-результатов.
    """
    print('\nИщем оптимальные параметры обучения\n')
    os.makedirs(results_dir, exist_ok=True)
    os.makedirs(os.path.join(results_dir, "best_models"), exist_ok=True)
    os.makedirs(os.path.join(results_dir, "plots"), exist_ok=True)

    # список всех вариантов в виде dict
    keys, values = zip(*param_grid.items())
    combos = [dict(zip(keys, v)) for v in itertools.product(*values)]

    all_records = []

    for idx, cfg in enumerate(combos, start=1):
        print(f"\nРассмотри {idx}/{len(combos)} – cfg: {cfg}\n")

        lr          = cfg["lr"]
        embed_dim   = cfg["embed_dim"]
        hidden_dim  = cfg["hidden_dim"]
        num_layers  = cfg["num_layers"]
        batch_size  = cfg["batch_size"]
        epochs      = cfg["epochs"]

        # пересоздаём даталоадеры, если меняется batch_size
        train_loader = DataLoader(
            base_train_loader.dataset,
            batch_size=batch_size,
            shuffle=True,
            collate_fn=token_collate_fn,
            num_workers=NUM_WORKERS,
            pin_memory=device.type == "cuda",
        )
        val_loader = DataLoader(
            base_val_loader.dataset,
            batch_size=batch_size,
            shuffle=False,
            collate_fn=token_collate_fn,
            num_workers=NUM_WORKERS,
            pin_memory=device.type == "cuda",
        )
        test_loader = DataLoader(
            base_test_loader.dataset,
            batch_size=batch_size,
            shuffle=False,
            collate_fn=token_collate_fn,
            num_workers=NUM_WORKERS,
            pin_memory=device.type == "cuda",
        )

        vocab_size = tokenizer.vocab_size
        model = LSTMWordGenerator(
            vocab_size=vocab_size,
            embed_dim=embed_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
        ).to(device)

        best_path = os.path.join(
            results_dir, "best_models", f"run_{idx:03d}_best.pt"
        )
        model, history = train(
            model,
            train_loader,
            val_loader,
            tokenizer,
            device,
            eos_id,
            epochs=epochs,
            lr=lr,
            warmup_ratio=0.0,         
            patience=3,
            best_model_path=best_path,
        )

        test_ppl, test_rouge, _ = evaluate_on_loader(
            model,
            test_loader,
            tokenizer,
            device,
            eos_id,
            fraction=0.5,
        )
        print(f"Test PPL: {test_ppl:.2f} |  Test ROUGE-L: {test_rouge:.4f}")

       # train_loss + val_rougeL
        fig1, ax1 = plt.subplots(figsize=(6, 4))
        ax1.plot(
            [h["epoch"] for h in history],
            [h["train_loss"] for h in history],
            label="train loss",
            marker="o",
            color="#1f77b4",
        )
        ax1.plot(
            [h["epoch"] for h in history],
            [h["val_rougeL"] for h in history],
            label="val ROUGE-L",
            marker="s",
            color="#ff7f0e",
        )
        ax1.set_xlabel("epoch")
        ax1.set_ylabel("value")
        ax1.set_title(f"Run {idx:03d} – train loss / val ROUGE-L")
        ax1.legend()
        plt.tight_layout()
        fig1.savefig(
            os.path.join(results_dir, "plots", f"run_{idx:03d}_loss_rouge.png")
        )
        plt.close(fig1)

        # train_loss + val_perplexity
        fig2, ax2 = plt.subplots(figsize=(6, 4))
        ax2.plot(
            [h["epoch"] for h in history],
            [h["train_loss"] for h in history],
            label="train loss",
            marker="o",
            color="#1f77b4",
        )
        ax2.plot(
            [h["epoch"] for h in history],
            [h["val_perplexity"] for h in history],
            label="val perplexity",
            marker="x",
            color="#2ca02c",
        )
        ax2.set_xlabel("epoch")
        ax2.set_ylabel("value")
        ax2.set_title(f"Run {idx:03d} – train loss / val perplexity")
        ax2.legend()
        plt.tight_layout()
        fig2.savefig(
            os.path.join(results_dir, "plots", f"run_{idx:03d}_loss_ppl.png")
        )
        plt.close(fig2)

        # аписываем строку в итоговую таблицу
        record = {
            **cfg,
            "final_train_loss": history[-1]["train_loss"],
            "final_val_perplexity": history[-1]["val_perplexity"],
            "final_val_rougeL": history[-1]["val_rougeL"],
            "test_perplexity": test_ppl,
            "test_rougeL": test_rouge,
            "best_model_path": best_path,
        }
        all_records.append(record)

        # сохраняем промежуточный CSV после каждой итерации (чтобы не потерять результаты)
        pd.DataFrame(all_records).to_csv(
            os.path.join(results_dir, "grid_search_results.csv"),
            index=False,
        )

    #возврат полной таблицы
    results_df = pd.DataFrame(all_records)
    results_df.to_csv(
        os.path.join(results_dir, "grid_search_results.csv"),
        index=False,
    )
    return results_df


def train(model: LSTMWordGenerator,
          train_loader: DataLoader,
          val_loader: DataLoader,
          tokenizer: AutoTokenizer,
          device: torch.device,
          eos_id: int,
          epochs: int = 10,
          lr: float = 5e-4,
          warmup_ratio: float = 0.1,
          patience: int = 2,
          best_model_path: str = 'best_blstm.pt') -> LSTMWordGenerator:
    '''
    Собирает историю метрик (train_loss, val_ppl, val_rouge) для каждой эпохи.

    Возвращает обученную модель (с лучшими весами) и список словарей-записей.
    '''
    total_steps = len(train_loader) * epochs
    # warmup_steps = int(warmup_ratio * total_steps)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    # scheduler = get_linear_schedule_with_warmup(
    #     optimizer,
    #     num_training_steps=total_steps,
    #     num_warmup_steps=warmup_steps
    # )

    best_val_rouge = -float('inf')
    no_improve = 0
    history = []

    # train
    for epoch in range(1, epochs + 1):
        model.train()
        epoch_loss = 0.0
        for batch in train_loader:
            optimizer.zero_grad()
            out = model(input_ids=batch['input_ids'].to(device),
                        attention_mask=batch['attention_mask'].to(device),
                        labels=batch['labels'].to(device))
            
            loss = out['loss']
            loss.backward()
            optimizer.step()
            # scheduler.step()

            epoch_loss += loss.item() * batch['input_ids'].size(0)
        avg_train_loss = epoch_loss / len(train_loader)

        # validation
        val_ppl, val_rouge, _ = evaluate_on_loader(model,
                                                val_loader,
                                                tokenizer,
                                                device,
                                                eos_id)
        print(f'\nEpoch {epoch:02d} | train_loss={avg_train_loss:.4f}'
              f' | valid. ppl={val_ppl:.2f} | valid.ROUGE-L={val_rouge:.4f}')
        history.append(
            {
                "epoch": epoch,
                "train_loss": avg_train_loss,
                "val_perplexity": val_ppl,
                "val_rougeL": val_rouge,
            }
        )
        # early stopping
        if val_rouge > best_val_rouge:
            best_val_rouge = val_rouge
            no_improve = 0
            torch.save(model.state_dict(), best_model_path)
        else:
            no_improve += 1
            if no_improve >= patience:
                break

    # загрузим лучшую
    model.load_state_dict(torch.load(best_model_path))
    return model, history


def train_final(best_cfg: dict,
                splits: DatasetDict,
                tokenizer: AutoTokenizer,
                device: torch.device,
                eos_id: int,
                final_model_path: str = 'full_final_model.pt',
                results_dir: str = 'models'):
    '''
    Объединяет train, val, test. обучает модель с лучшими гиперпараметрами
    и сохраняет её в <results_dir>/final_model.pt.
    Возвращает обученную модель.
    '''
    print('\nОбучение финальной модели...\n')
    full_dataset = concatenate_datasets([splits["train"],
                                         splits["validation"],
                                         splits["test"]])

    full_tweet_ds = TweetDataset(full_dataset, eos_id=eos_id)
    batch_size = best_cfg["batch_size"]
    # Даталоадер (только train-loader, валидации нет)
    train_loader = DataLoader(
        full_tweet_ds,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=token_collate_fn,
        num_workers=NUM_WORKERS,
        pin_memory=device.type == "cuda",
    )
    # Инициализируем модель с найденными гиперпараметрами
    vocab_size = tokenizer.vocab_size
    model = LSTMWordGenerator(
        vocab_size=vocab_size,
        embed_dim=best_cfg["embed_dim"],
        hidden_dim=best_cfg["hidden_dim"],
        num_layers=best_cfg["num_layers"]).to(device)

    # обучаем с использованием early-stopping для защиты от переобучения
    # но теперь мониторим ROUGE-L на 10% от полной выборки
    split_tmp = full_dataset.train_test_split(test_size=0.10, seed=42)
    val_hf = split_tmp["test"]
    val_ds = TweetDataset(val_hf, eos_id=eos_id)
    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=token_collate_fn,
        num_workers=NUM_WORKERS,
        pin_memory=device.type == "cuda",
    )

    epochs = best_cfg["epochs"] * 2 # удваиваем чтобы дать модели шанс разойтись на большом объёме, но early-stop всё равно прервет
    lr = best_cfg["lr"]
    patience = 3

    model, _ = train(
        model,
        train_loader=train_loader,
        val_loader=val_loader,
        tokenizer=tokenizer,
        device=device,
        eos_id=eos_id,
        epochs=epochs,
        lr=lr,
        warmup_ratio=0.0,         
        patience=patience,
        best_model_path=os.path.join(results_dir, "best_models/final_model.pt")
    )

    torch.save(model, os.path.join(results_dir, f"best_models/{final_model_path}"))

    print(f"\nВеса итоговой модели сохранены в {os.path.join(results_dir, 'final_model.pt')}")
    print(f"\nПолная финальная модель сохранена в {os.path.join(results_dir, 'full_final_model.pt')}")
    return model


def train_lstm(tokenizer: AutoTokenizer,
               eos_id: int):
    # Поиск оптимальных параметров обучения
    grid = {
        "lr":          [5e-4],
        "embed_dim":   [128],
        "hidden_dim":  [128],
        "num_layers":  [1],
        "batch_size":  [16],            
        "epochs":      [4],
    }

    # общие настройки ----------
    set_seed(2025)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Чтение и токенизация ----------
    # MAX_LEN  = 128
    NROWS    = None if device.type == "cuda" else 50              
    hf, cleaned_texts = read_dataset(CSV_PATH, 
                                     tokenizer, 
                                     max_length=tokenizer.model_max_length, 
                                     nrows=NROWS)

    # делим на сплиты ----------
    splits = split_hf_dataset(
        hf, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, seed=2025
    )

    # даталоадеры будут пересозданы внутри grid_search
    # они нужны только чтобы взять `dataset` и параметры `num_workers/pin_memory`
    base_train_ds = TweetDataset(splits["train"], eos_id=eos_id) 
    base_val_ds   = TweetDataset(splits["validation"], eos_id=eos_id)
    base_test_ds  = TweetDataset(splits["test"], eos_id=eos_id)
    base_train_loader = DataLoader(
        base_train_ds,
        batch_size=64,
        shuffle=True,
        collate_fn=token_collate_fn,
        num_workers=NUM_WORKERS,
        pin_memory=device.type == "cuda",
    )
    base_val_loader = DataLoader(
        base_val_ds,
        batch_size=64,
        shuffle=False,
        collate_fn=token_collate_fn,
        num_workers=NUM_WORKERS,
        pin_memory=device.type == "cuda",
    )
    base_test_loader = DataLoader(
        base_test_ds,
        batch_size=64,
        shuffle=False,
        collate_fn=token_collate_fn,
        num_workers=NUM_WORKERS,
        pin_memory=device.type == "cuda",
    )

    # Запуск grid-search
    results_df = grid_search(
        param_grid=grid,
        base_train_loader=base_train_loader,
        base_val_loader=base_val_loader,
        base_test_loader=base_test_loader,
        tokenizer=tokenizer,
        device=device,
        eos_id=eos_id,
        results_dir="models",
    )

    # Находим лучшую конфигурацию (по тестовому ROUGE-L)
    best_cfg = results_df.loc[results_df["test_rougeL"].idxmax()].to_dict()
    print("\nЛучшая модель (по test-rougeL)")
    print(best_cfg)
    
    # Обучение финальной модели (без сплита данных)
    final_model = train_final(best_cfg=best_cfg,
                              splits=splits,
                              tokenizer=tokenizer,
                              device=device,
                              eos_id=eos_id,
                              results_dir='models')
    return final_model

# Сетап инференса LSTM & Transformer

In [7]:
def inference_lstm(*,
                   tokenizer: AutoTokenizer,
                   eos_id: int,
                   use_sampling: bool = False,
                   temperature: float = 1.0,
                   top_p: float = 0.9):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # best_lstm_path = "./models/best_models/full_final_model.pt" 
    # model = torch.load(best_lstm_path, map_location=device) # полная модель не хочет загружаться:
    '''
    AttributeError: Can't get attribute 'LSTMWordGenerator' 
    on <module '__main__' from '/Users/dotsenko.a.v/yandex/sprint2-project/src/compare.py'>
    '''
    vocab_size = tokenizer.vocab_size
    model = LSTMWordGenerator(
        vocab_size=vocab_size,
        embed_dim=128,
        hidden_dim=128,
        num_layers=1).to(device)
    model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device))
    model.eval()

    # Формируем DataLoader для validation-части.
    val_loader, hf, val_texts = make_val_loader(tokenizer=tokenizer,
                                                    eos_id=eos_id,
                                                    device=device,
                                                    nrows=COMPARISON_DS_SIZE)

    # Вычисляем метрики. verbose=True заставит функцию вернуть
    #      список ROUGE-L по каждому примеру.
    ppl, avg_rouge, per_example = evaluate_on_loader(model=model,
                                                        loader=val_loader,
                                                        tokenizer=tokenizer,
                                                        device=device,
                                                        eos_id=eos_id,
                                                        fraction=0.5,
                                                        verbose=True,
                                                        use_sampling=use_sampling,
                                                        temperature=temperature,
                                                        top_p=top_p)

    val_hf = hf
    autocompleted_texts = []
    for i in range(len(val_hf)):
        ids = val_hf[i]["input_ids"]
        # убираем PAD (0) и EOS, если они есть
        if 0 in ids:
            ids = ids[:ids.index(0)]
        if eos_id in ids:
            ids = ids[:ids.index(eos_id)]
        # получаем чистый текст твита
        src_text = tokenizer.decode(ids, skip_special_tokens=True)
        # генерируем продолжение (используем уже обученную модель)
        fraction = 0.5
        split_idx = int(len(src_text) * fraction)
        prompt_text = src_text[:split_idx] 
        compl = model.generate(
            prompt_ids=tokenizer.encode(prompt_text, add_special_tokens=False),
            eos_id=eos_id,
            do_sampling=use_sampling,
            temperature=temperature,
            top_p=top_p,
        )
        compl_text = tokenizer.decode(compl, skip_special_tokens=True)
        autocompleted_texts.append({'prompt': prompt_text, 
                                    'prediction': compl_text})

    return avg_rouge, autocompleted_texts


def generate_completion(prompt: str,
                        tokenizer: AutoTokenizer,
                        model: AutoModelForCausalLM,
                        temperature: float = 1.0,
                        top_p: float = 0.9,
                        do_sample: bool = False) -> str:
    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
        )
    generated = out_ids[0][inputs["input_ids"].shape[-1] :]
    return tokenizer.decode(generated, skip_special_tokens=True)


def validate_on_dataset(texts: list[str],
                        tokenizer: AutoTokenizer,
                        model: AutoModelForCausalLM,
                        fraction: float = 0.5,
                        do_sampling: bool = False,
                        temperature: float = 1.0,
                        top_p: float = 0.9) -> dict:
    """
    Для каждого текста берём первые 75 % как prompt,
    а оставшиеся – как reference.
    """
    f1_scores = []
    examples = []

    for txt in texts:
        split_idx = int(len(txt) * fraction)
        prompt, reference = txt[:split_idx], txt[split_idx:]

        pred = generate_completion(
            prompt,
            tokenizer=tokenizer,
            model=model,
            temperature=temperature,
            top_p=top_p,
            # switch between greedy / sampling
            do_sample=do_sampling
        )

        score = rouge_l_f1(reference, pred)
        f1_scores.append(score)

        examples.append({
            "prompt":      prompt,
            "prediction":  pred
        })

    avg_f1 = float(np.mean(f1_scores)) if f1_scores else 0.0
    return {"rougeL_f1": avg_f1, "examples": examples}

def inference_transformer(*,
                          transformer_name: str,
                          tokenizer: AutoTokenizer,
                          eos_id: int,
                          use_sampling: bool = False,
                          temperature: float = 1.0,
                          top_p: float = 0.9):
    model = AutoModelForCausalLM.from_pretrained(transformer_name)
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        model = model.to(device)

    # Получаем готовый DataLoader и **тексты** для трансформера
    val_loader, hf, val_texts = make_val_loader(
        tokenizer=tokenizer,          
        eos_id=eos_id,
        device=device,
        nrows=COMPARISON_DS_SIZE,                     
    )

    # Валидация трансформера (ROUGE-L F1)
    results = validate_on_dataset(
                texts=val_texts,
                tokenizer=tokenizer,
                model=model,
                fraction=0.5,
                # передаём параметры в generate()
                do_sampling=use_sampling,
                temperature=temperature,
                top_p=top_p,
    )
    avg_rouge = results['rougeL_f1']
    # autocompleted_texts = [{'prompt': ex['prompt'], 
    #                         'prediction': ex['prediction']} for i, ex in enumerate(results["examples"], 1)]
    autocompleted_texts = results["examples"] 
    return avg_rouge, autocompleted_texts

# Переходим к экспериментам: можно пропустить ячейку с обучением LSTM для проверки весов, полученных на VM. Потом для проверки работы обучения можно раскоментировать и запустить

In [8]:
# надо понять какой токенизатор у трансформера
TRANSFORMER_NAME = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_NAME)
eos_id = get_eos_id(tokenizer)

# Обучение LSTM (если раскоментировать train_lstm, то модель из репы перезатрется)

In [9]:
# обучаем с этим токенизатором
# model = train_lstm(tokenizer=tokenizer, eos_id=eos_id)

# Попытки автодополнения

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


vocab_size = tokenizer.vocab_size
model = LSTMWordGenerator(
    vocab_size=vocab_size,
    embed_dim=128,
    hidden_dim=128,
    num_layers=1).to(device)

model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device))
model.eval()



initial_text = "Moscow is"

print(f'LSTM локанично сказала: {initial_text}', model.generate_one_word(initial_text, tokenizer))

print(f'LSTM разговорилась: {initial_text}', model.generate_n_words(initial_text, n=5, tokenizer=tokenizer))

LSTM локанично сказала: Moscow is  sick
LSTM разговорилась: Moscow is sick!!!!


# Сравнение скорингов

In [10]:
# Жадная генерация
mean_rouge_lstm_greedy, lstm_greedy_texts = inference_lstm(
                                    tokenizer=tokenizer,
                                    eos_id=eos_id,
                                    use_sampling=False)

mean_rouge_transformer_greedy, transformer_greedy_texts = inference_transformer(
                                            transformer_name=TRANSFORMER_NAME,
                                            tokenizer=tokenizer,
                                            eos_id=eos_id,
                                            use_sampling=False)


temperature=0.8
top_p=0.9
mean_rouge_lstm_sampling, lstm_sampling_texts = inference_lstm(
                                        tokenizer=tokenizer,
                                        eos_id=eos_id,
                                        use_sampling=True,
                                        temperature=temperature,
                                        top_p=top_p)

mean_rouge_transformer_sampling, transformer_sampling_texts = inference_transformer(
                                            transformer_name=TRANSFORMER_NAME,
                                            tokenizer=tokenizer,
                                            eos_id=eos_id,
                                            use_sampling=True,
                                            temperature=temperature,
                                            top_p=top_p)

Map: 100%|██████████| 100/100 [00:00<00:00, 8074.67 examples/s]
The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The following generation flags are not valid and may be ignored: ['top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting 

# Интерпретация результатов

In [11]:
print(f'{mean_rouge_lstm_greedy=}')
print(f'{mean_rouge_transformer_greedy=}')
print(f'{mean_rouge_lstm_sampling=}')
print(f'{mean_rouge_transformer_sampling=}')

mean_rouge_lstm_greedy=0.07065358683050327
mean_rouge_transformer_greedy=0.051920717352681785
mean_rouge_lstm_sampling=0.06695747807311582
mean_rouge_transformer_sampling=0.05895486075012723


In [12]:
print('LSTM с семплированием:')
for j in range(5):
    print(f'PROMT: {lstm_sampling_texts[j]["prompt"]}')
    print(f'PREDICTION: {lstm_sampling_texts[j]["prediction"]}\n')

LSTM с семплированием:
PROMT: - awww, that's a bummer. you shoulda g
PREDICTION: - awww, that's a bummer. you shoulda gals! but i'm tired!!!!!!!!!! no!!!

PROMT: is upset that he can't update his facebook by texting 
PREDICTION: is upset that he can't update his facebook by texting ive a little!.!!, but not having a tummy ache!!!!

PROMT: i dived many times for the ball. manage
PREDICTION: i dived many times for the ball. manage to get the front-it's so was drunk!!!!! ...!!!s

PROMT: my whole body feels itc
PREDICTION: my whole body feels itc!!!! is only rude and i want to cry...!! the opportunity is wemble

PROMT: no, it's not behaving at all. i'm mad. why am 
PREDICTION: no, it's not behaving at all. i'm mad. why am ive been procrastinating anymore! i hate that it has been worse than so!!!!



In [13]:
print('TRANSFORMER с семплированием:')
for j in range(5):
    print(f'PROMT: {transformer_sampling_texts[j]["prompt"]}')
    print(f'PREDICTION: {transformer_sampling_texts[j]["prediction"]}\n')

TRANSFORMER с семплированием:
PROMT: - awww, that's a bummer. you shoulda g
PREDICTION: iddy if it's a bit of a surprise. The bummer is that it's the third time

PROMT: is upset that he can't update his facebook by texting i
PREDICTION:  had to do it and after I have seen that he is not posting anything to his friends. I

PROMT: i dived many times for the ball. manage
PREDICTION:  to stay in the back of the net.












PROMT: my whole body feels itc
PREDICTION: 

The thing is, I feel very, very happy with this body. I feel pretty good

PROMT: no, it's not behaving at all. i'm mad. why am 
PREDICTION: icky? i'm not mad. why am icky? i'm not mad. why am 



In [14]:
print('LSTM жадная генерация:')
for i in range(5):
    print(f'PROMT: {lstm_greedy_texts[i]["prompt"]}')
    print(f'PREDICTION: {lstm_greedy_texts[i]["prediction"]}\n')

LSTM жадная генерация:
PROMT: - awww, that's a bummer. you shoulda g
PREDICTION: - awww, that's a bummer. you shoulda gd you!!!!!!!!!!!!!!!!!!

PROMT: is upset that he can't update his facebook by texting 
PREDICTION: is upset that he can't update his facebook by texting !!!!!!!!!!!!!!!!!!!!!!

PROMT: i dived many times for the ball. manage
PREDICTION: i dived many times for the ball. manage to get a new one.!!!!!!!!!!!!!!

PROMT: my whole body feels itc
PREDICTION: my whole body feels itc!!!!!!!!!!!!!!!!!!!!

PROMT: no, it's not behaving at all. i'm mad. why am 
PREDICTION: no, it's not behaving at all. i'm mad. why am ive been up since 4am and i'm so tired!!!!!!!!!



In [17]:
print('TRANSFORMER жадная генерация:')
for i in range(5):
    print(f'PROMT: {transformer_greedy_texts[i]["prompt"]}')
    print(f'PREDICTION: {transformer_greedy_texts[i]["prediction"]}\n')

TRANSFORMER жадная генерация:
PROMT: - awww, that's a bummer. you shoulda g
PREDICTION: osh, but I'm not going to be able to do that. I'm going to be able

PROMT: is upset that he can't update his facebook by texting i
PREDICTION: Message.



















PROMT: i dived many times for the ball. manage
PREDICTION:  to get the ball out of the box.












PROMT: my whole body feels itc
PREDICTION: oughing.”
















PROMT: no, it's not behaving at all. i'm mad. why am 
PREDICTION: ive seen this?


I'm not sure if it's a good idea to just say



# Что лучше использовать

Судя по метрикам, обученная модель LSTM лучше справляется задачей как на жадной генерации, так и при семплировании. Если посмотреть на сами сгенерованные тексты, то трансформер может выдавать такую девиацию как множество последовательных переносов строк, что совсем не характерно для твитов. В целом, обе модели оставляют желать лучшего. Чтобы эксперимент был честным, стоило бы дообучить трансформер

Вывод: для автогенерации твитов текущая обученная LSTM работает немного лучше чем трансформер distilbert (без дообучения). Видимо, трансформер был обучен на совершенно других текстах.